In [ ]:
import praw
import time
import pandas as pd
from datetime import datetime
import re  

# Reddit API setup
reddit = praw.Reddit(
    client_id="j9JdlLnSugDJxrhTFoXExQ",
    client_secret="QJbefKRgxmm7fY1ZYp6oQL0iv2wgUg",
    user_agent="Hi",
    username="FRD-23",
    password="@_Zurite1721212232005"
)

# Function to fetch EPL match threads from matchday 20 onward
def get_matchday_threads(year, start_matchday=20):
    redditor = reddit.redditor("MatchThreadder")  
    match_threads = []  
    seen_matches = set()  # Track unique matches to avoid duplicates

    print(f"Searching for EPL match threads from matchday {start_matchday} in {year}...")

    for submission in redditor.submissions.new(limit=1000):
        try:
            post_time = datetime.utcfromtimestamp(submission.created_utc)
            post_year = post_time.year  

            if post_year != year:
                continue  

            title_lower = submission.title.lower()
            matchday_search = re.search(r"matchday (\d+)", title_lower)
            if matchday_search:
                matchday_number = int(matchday_search.group(1))
                if matchday_number < start_matchday:
                    continue  
            
            if "match thread" in title_lower and "premier league" in title_lower:
                match_title = submission.title  # Use full title for uniqueness check
                if match_title in seen_matches:
                    continue  
                seen_matches.add(match_title)
                
                match_score = extract_match_score(submission.title)
                print(f"Found EPL Thread: {submission.title} {post_time} | {submission.url}")
                match_threads.append({
                    "Title": submission.title,
                    "URL": submission.url,
                    "Date": post_time.strftime('%Y-%m-%d'),
                    "Score": match_score
                })
        except Exception as e:
            print(f"Error processing post: {e}")

    return match_threads  

# Function to extract match points (final score) from the title
def extract_match_score(title):
    match = re.search(r"(\d+)\s*-\s*(\d+)", title)
    if match:
        return f"{match.group(1)}-{match.group(2)}"  
    return "Unknown"

# Function to scrape comments from a match thread
def scrape_comments(match_threads):
    all_comments = []

    for match in match_threads:
        post_url = match["URL"]
        print(f"\nScraping comments for: {match['Title']}")

        try:
            submission = reddit.submission(url=post_url)
            submission.comments.replace_more(limit=3)  

            for comment in submission.comments.list():
                try:
                    time.sleep(1.5)  
                    all_comments.append({
                        "Match Date": match["Date"],
                        "Match Title": match["Title"],
                        "Score": match["Score"],
                        "Comment": comment.body,
                        "Score (Votes)": comment.score,
                        "Author": comment.author.name if comment.author else "[deleted]",
                        "Timestamp": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    })
                except Exception as e:
                    print(f"Error processing comment: {e}")
        except Exception as e:
            print(f"Error fetching post: {e}")

    return all_comments

# Input: Year to scrape
year = 2025  
matchday_start = 20  

# Get match threads from matchday 20 onward
match_threads = get_matchday_threads(year, matchday_start)

if match_threads:
    comments_data = scrape_comments(match_threads)
    df_matches = pd.DataFrame(match_threads)
    csv_filename_matches = f"epl_match_threads_{year}_matchday{matchday_start}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_matches.to_csv(csv_filename_matches, index=False)
    print(f"\nExported {len(match_threads)} EPL match threads from matchday {matchday_start} onward to {csv_filename_matches}")

    if comments_data:
        df_comments = pd.DataFrame(comments_data)
        csv_filename_comments = f"epl_match_comments_{year}_matchday{matchday_start}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_comments.to_csv(csv_filename_comments, index=False)
        print(f"\nExported {len(comments_data)} comments to {csv_filename_comments}")
    else:
        print("\nNo comments extracted.")
else:
    print(f"\nNo EPL match threads found from matchday {matchday_start} onward in {year}.")